# Intro RL para LLMs

- Agent/policy model: Estrategia para tomar decisiones. Es el LLM
- Environment: provee feedback al agente. Puede ser muchas cosas aca
- Action: Pueden ser PALABRAS, RESPUESTA ENTERA, OUTCOME FINAL, etc
- Reward: Feedback que le da el env al agent. Numero

---

**RLHF**

- Preferencias humanas: a partir de varias respuestas, las ranqueamos
- Reward model: construimos un reward model basado en esas preferencias. Ahora tenemos un modelo que, dado una respuesta, estima el puntaje que le darian humanos
- Fine-tune el LLM con RL: Ahora el reward model nos da el feedback. Entonces:
    - El agente genera respuestas
    - El reward model las scorea
    - Ajustamos los weights para que tienda a producir outputs que sean bien puntuados


---

**RL vs SL**

Principalmente cambia la fuente de la señal (reward). Qué optimizas realmente
- SL: Minimizar divergencia respecto a la distribución objetivo (imitar). Tiene un reward **denso** porque sabes la respuesta target para cada token.
- RL: Maximizar retorno esperado bajo dinámicas del entorno (elegir). La señal es escasa y esta diferida en el tiempo. Se requiere **exploracion** y credito temporal. 

Por eso se dice que SFT memorizes y RL generalizes https://arxiv.org/abs/2501.17161

---



# Policy optimization (gradient-based)

Tecnicas para optimizar el LLM / policy model para que mejore las predicciones.

**Intro**:

En SL tenemos una funcion de perdida (loss) y lo que queremos es minimizar esa loss. Entonces dado un batch, calculamos la loss (un numero final) basado en muchos errores (1 o varios para cada elemento del batch), los promediamos y nos queda ese numero final. Ahi, buscamos el gradiente (el conjunto de derivadas parciales de cada uno de los parametros aprendibles) que hagan que mas baje la loss en ese punto. Y actualizamos los valores de los parametros en la direccion y fuerza de esas derivadas y agregandole otros empujones.

En RL, lo que tratamos de hacer es subir o bajar la probabilidad de los tokens segun si aparecen en la secuencia que genero un buen o mal resultado (bien o mal comparado con un baseline).


**Pasos general (on-policy)**:
- Se hacen rollouts (el modelo genera respuestas completas)
- Se evalua cada respuesta y se obtiene score o reward
- El score se convierte en ventaje (advantage): cuanto mejor o peor esta respuesta fue a lo esperado o tipico (por ej del grupo o de algo de referencia).
    - Si tengo solo reward final, el advantage se aplica a todos los tokens de la secuencia de la misma manera (credito global)
    - Si tengo verificadores por paso, se reparten ventajas distintas por token
- Distribuyo la ventaja para cada token:
    - Si es positiva, subo su log-prob (empujo logits para arriba).
    - Si es negativa, la bajo.
    - Indirectamnete afecto a las otras (porque en probs la suma = 1)
- Regularizo para no irme tan lejos de una distribucion anterior (basada en una politica de referencia) para evitar colapso.

**Caso general**
- Nosotros tenemos una red y para un estado (secuenci), produce logits (next token / action)
- Los pasamos por softmax y tenemos probs y de esos tomamos el log, por lo que trabajamos con log probs. 
- Para el token elegido en un paso t, vemos el log prob (solo tenemos un log prob en ese paso)
- Si tomamos el gradiente de ese log prob, estamos viendo un conjunto de derivadas parciales del logprob con respecto a todos los parametros del modelo. Esto indica la direccion de MAXIMA SUBIDA -> o sea, si tomamos un paso en esa direccion, aumentamos la logprob de ese token para la prox.
- Si hacemos lo mismo para cada token de la secuencia de pasos generados, y sumamos los gradientes (sumamos las derivadas parciales (o sea, para un parametro miramos las derivadas parciales por cada token en la secuencia generada y los sumamos)), nos queda un gradiente unico que apunta a maximizar todos los tokens en la secuencia generada. 
- Despues vemos el reward para esa secuencia y lo comparamos con el baseline (R-b). Si el baseline es 0.5 y nuestro reward es 0.2, el advantage da -0.3. Eso lo usamos para multiplicar, lo que nos cambia la direccion del gradiente si el advantage es negativo y tambien nos da una magnitud de cambio (escala el tamaño del paso junto con el lr). O sea que si es negativo, en vez de apuntar a aumentar la prob, apunta a disminuir la prob de los tokens.
- Para no favorecer secuencias largas, a veces se promedia la suma de logprobs y se suele agregar entropia (para mantener diversidad) y KL a un modelo de referencia para regularizar.


## REINFORCE

$$\text L_{(REINFORCE)} = - (R - b) \sum_t \log \pi_\theta(a_t \mid s_t)$$

$$\text L_{(REINFORCE)} = - \sum_t (R - b) \log \pi_\theta(a_t \mid s_t)$$

El gradiente para la secuencia entera seria:

$$\Delta \theta \propto (R - b) \sum_t \nabla_\theta \log \pi_\theta(a_t \mid s_t)$$



In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

torch.manual_seed(0)

vocab_size=5
T=3
lr=0.5

# Model: linear map from one-hot(state) to vocab logits (bias-free for clarity).
model = nn.Linear(T, vocab_size, bias=False)
# Initialize weights small & reproducible
with torch.no_grad():
    model.weight.copy_(0.1 * torch.randn(vocab_size, T))

opt = optim.SGD(model.parameters(), lr=lr)

log_softmax = nn.LogSoftmax(dim=-1)

model.weight

Parameter containing:
tensor([[ 0.0604,  0.0811, -0.0045],
        [ 0.0880,  0.1048, -0.0045],
        [-0.0723,  0.2866, -0.0566],
        [ 0.0160, -0.0025,  0.1074],
        [ 0.2263, -0.0918, -0.0225]], requires_grad=True)

In [31]:
# Inputs: one-hot vectors for each time step 0..T-1
I = torch.eye(T)  # shape (T, T)
I

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [32]:
# CASO 1 (GOOD)
chosen_tokens = [2, 1, 4]
advantage = 0.4

# --- Forward BEFORE update: record chosen tokens' log-probs and probs ---
with torch.no_grad():
    before = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        before.append((tok, float(logp_t[tok]), float(logp_t.exp()[tok])))
before

[(2, -1.750232458114624, 0.17373354732990265),
 (1, -1.5883560180664062, 0.20426113903522491),
 (4, -1.6373724937438965, 0.19449038803577423)]

In [33]:
# --- Compute REINFORCE loss:  L = -A * sum_t log pi(a_t | s_t) ---
total_logprob = 0.0
for t in range(T):
    # sumamos todos los logprobs de los tokens elegidos
    logits_t = model(I[t])          # (vocab,)
    logp_t  = log_softmax(logits_t) # (vocab,)
    tok = chosen_tokens[t]
    total_logprob = total_logprob + logp_t[tok]
    print(f't={t}: token {tok} logprob={logp_t[tok]} total_logprob={total_logprob}')
print(f'-advantage={-advantage}\n-advantage * total_logprob = {-advantage} - {total_logprob}')
loss = - advantage * total_logprob
loss

t=0: token 2 logprob=-1.750232458114624 total_logprob=-1.750232458114624
t=1: token 1 logprob=-1.5883560180664062 total_logprob=-3.3385884761810303
t=2: token 4 logprob=-1.6373724937438965 total_logprob=-4.975960731506348
-advantage=-0.4
-advantage * total_logprob = -0.4 - -4.975960731506348


tensor(1.9904, grad_fn=<MulBackward0>)

In [34]:
# Backprop + step
opt.zero_grad()
loss.backward()
opt.step()

model.weight

Parameter containing:
tensor([[ 0.0207,  0.0412, -0.0441],
        [ 0.0472,  0.2640, -0.0441],
        [ 0.0930,  0.2376, -0.0941],
        [-0.0219, -0.0392,  0.0631],
        [ 0.1794, -0.1253,  0.1386]], requires_grad=True)

In [37]:
# --- Forward AFTER update: record chosen tokens' log-probs and probs ---
with torch.no_grad():
    after = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        after.append((tok, float(logp_t[tok]), float(logp_t.exp()[tok])))

print("Per-step chosen token probabilities (before -> after):")
for t, ((tok_b, logpb, pb), (tok_a, logpa, pa)) in enumerate(zip(before, after)):
    assert tok_b == tok_a
    arrow = "↑" if pa > pb else ("↓" if pa < pb else "→")
    print(f"  t={t}: token={tok_b}   P_before={pb:.4f} -> P_after={pa:.4f}   ({arrow})")

Per-step chosen token probabilities (before -> after):
  t=0: token=2   P_before=0.1737 -> P_after=0.2055   (↑)
  t=1: token=1   P_before=0.2043 -> P_after=0.2386   (↑)
  t=2: token=4   P_before=0.1945 -> P_after=0.2280   (↑)


## PPO

Limita cuanto se puede cambiar la distribucion para que no hayan pasos demasiado grandes y colapse.

Hay un ratio por token

$$r_t = \frac{\pi_{\text{nueva}}(a_t \mid s_t)}{\pi_{\text{vieja}}(a_t \mid s_t)}$$

Que te dice cuanto cambiaste la prob para un token. Si es 1.3, subiste 30% la probabilidad para ese token. Si es 0.7 bajaste 30% la prob para ese token.

- **Clipping**: Si el paso te lleva a un r_t demasiado grande > 1 + eps, entonces recorto el beneficio
- **KL divergence a un modelo referencia**: Penaliza alejarte del modelo base de referencia (el original) en promedio.
- **Ventaja por paso** (opcional): en vez de tener un unico A global, lo hacemos por paso, con V(s_t) para asignar mejor el credito y premiar a los tokens clave. Si no esta, se puede usar como baseline la media del batch.
- **Entropia**: bonus que mantiene la exploracion y evita colapso de entropia (que se vuelva modo unico)

**PPO Loss**:

$$L = \mathbb{E}\left[ \min\left(r_t A_t, \; \text{clip}(r_t, 1-\epsilon, 1+\epsilon) A_t \right) \right] - \beta \, \mathrm{KL}(\pi_{\text{new}} \| \pi_{\text{ref}}) + \alpha \, \text{Entropía}$$

Tres grandes partes:

- Politica -> $\mathbb{E}\left[ \min\left(r_t A_t, \; \text{clip}(r_t, 1-\epsilon, 1+\epsilon) A_t \right) \right]$
- KL -> $- \beta \, \mathrm{KL}(\pi_{\text{new}} \| \pi_{\text{ref}})$
- Entropia -> $\alpha \, \text{Entropía}$


Politica

$r_t = \frac{\pi_{\text{new}}(a_t \mid s_t)}{\pi_{\text{old}}(a_t \mid s_t)} = \exp\left( \log \pi_{\text{new}}(a_t \mid s_t) - \log \pi_{\text{old}}(a_t \mid s_t) \right)$

- $\log \pi_{\text{new}}(a_t \mid s_t)$ -> se hace backprop
- $\log \pi_{\text{old}}(a_t \mid s_t)$ -> constante

Como en REINFORCE, aca tambien aparece el **logprob $\log \pi(a_t \mid s_t)$. Por aca fluye el gradiente**.

PPO es REINFORCE con:
- Un peso r_t que te dice cuanto ya cambiaste la prob
- Un clip que apaga el grad si te pasaste. Si el clip se activa (te pasaste), el término se vuelve constante → gradiente 0 (no empuja más ese token en este minibatch). El clip define una ZONA segura: por ej [0.8, 1.2], entonces el ratio siempre deberia estar ahi adentro. Si r_t sale de esa zona, apagamos el grad.


**Gradiente**: Cuando usamos r_t, tenemos la propiedad que el gradiente del ratio es el grad del logprob_new multiplicado por el valor posta del ratio

- $\nabla r = r \cdot \nabla \log \pi_{\text{new}}$

Entonces el gradiente (solo de policy y sin usar CLIP), simplificado, queda:

$$\Delta \theta \propto \sum_t A \cdot r_{t} \cdot \nabla \log \pi_{\text{new}}$$ 

Entonces, cuando r_t = 1 (al principio cuando las dos policies son iguales), entonces el gradiente es igual a REINFORCE!

**El ratio solo pondera** y permite clippear, pero en cuanto a grad, es lo mismo que reinforce casi.


---

**FLUJO**

Los policies new NO SAMPLEAN. Lo unico que samplea es el OLD (se va actualizando cada tanto).

- Tenemos un modelo pi_old. Tenemos prompts. Creamos minibatches (por ejemplo 4 prompts en cada minibatch) y tenemos 3 minibatches (12 ejemplos en total). 
- Sampleamos la respuesta usando pi_old y guardamos toda la info (probs de los tokens para CADA step, reward, etc) para cada prompt en todos los minibatches.
- Ahora empieza la primera corrida de varios EPOCH (1 epoch = 1 pasada por todos los minibatches). Esto se va a hacer SIN HACER NINGUN SAMPLEO A MODELOS. Solo con los datos que estan.
- Agarramos el primer minibatch de 4 ejemplos. Para cada ejemplo ya tenemos la secuencia/respuesta originada por el pi_old. Agarramos el primer ejemplo y vamos token por token de la secuencia guardad por el pi_old y vamos comparando los tokens seleccionados con la prob que arroja el modelo (o sea vamos usando el modelo pi_new para ver la prob que arroja para los tokens muestrados originalmente por el pi_old). Obviamente en la primera vuelta va a ser igual o casi igual que el pi_old porque es el mismo modelo, todavia no se actualizo. Usamos la formula de PPO, sumamos para cada token en la secuencia. -> **El advantage despues lo veo bien**
- Lo mismo para cada ejemplo del batch. Y promediamos para cada ejemplo del batch.
- Despues de cada minibatch, backprop y optimizamos!
- Seguimos con los minibatches hasta terminar todos, hasta ahi 1 epoch
- Hacemos los varios epochs lo mismo. OJO, el pi_new va acumulando los cambios PERO NO VA SAMPLEANDO.
- DESPUES DE ESO, volvemos a samplear pero con el pi_new que reemplaza al pi_old.

**PPO simplificado** (solo parte de policy clipped. Sin KL ni entropy)

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)

# Hiperparámetros chiquitos
vocab_size = 5
T = 3
lr = 0.5
eps_clip = 0.2   # zona segura [0.8, 1.2]
advantage = 0.4  # mismo A para todos los pasos (reward final - baseline)

# Modelo lineal sin bias (como el tuyo)
model = nn.Linear(T, vocab_size, bias=False)
with torch.no_grad():
    model.weight.copy_(0.1 * torch.randn(vocab_size, T))

opt = optim.SGD(model.parameters(), lr=lr)
log_softmax = nn.LogSoftmax(dim=-1)

# Estados one-hot (t = 0..T-1)
I = torch.eye(T)

# Elegimos una secuencia de tokens (simula lo que generó π_old)
chosen_tokens = [2, 1, 4]

In [57]:
# ----------------------------------------------------------
# 1) "Foto" de π_old: guardamos logprob_old en esos (s_t, a_t)
# ----------------------------------------------------------
with torch.no_grad():
    logprob_old = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        logprob_old.append(float(logp_t[tok]))
    logprob_old = torch.tensor(logprob_old, dtype=torch.float32)

print("logprob_old por paso:", [f"{x:.4f}" for x in logprob_old.tolist()])

logprob_old por paso: ['-1.7502', '-1.5884', '-1.6374']


In [58]:
# ----------------------------------------------------------
# 2) BEFORE update: logprob_new y ratio (r_t ≈ 1 al inicio)
# ----------------------------------------------------------
with torch.no_grad():
    before = []
    ratio_before = []
    for t in range(T):
        logits_t = model(I[t])
        logp_t  = log_softmax(logits_t)
        tok = chosen_tokens[t]
        logp_new = logp_t[tok]
        r_t = torch.exp(logp_new - logprob_old[t])
        before.append((tok, float(logp_new), float(logp_t.exp()[tok])))
        ratio_before.append(float(r_t))
print("logprob_new BEFORE:", [f"{x[1]:.4f}" for x in before])
print("P_new BEFORE      :", [f"{x[2]:.4f}" for x in before])
print("r_t BEFORE        :", [f"{r:.4f}" for r in ratio_before])

logprob_new BEFORE: ['-1.7502', '-1.5884', '-1.6374']
P_new BEFORE      : ['0.1737', '0.2043', '0.1945']
r_t BEFORE        : ['1.0000', '1.0000', '1.0000']


In [59]:
# ----------------------------------------------------------
# 3) Loss PPO (sólo política, clipeada)
#    L = - mean( min( r_t*A, clamp(r_t)*A ) )
# ----------------------------------------------------------
advantages = torch.full((T,), fill_value=advantage, dtype=torch.float32)

total_unclipped = 0.0
total_clipped = 0.0
for t in range(T):
    logits_t = model(I[t])
    logp_t  = log_softmax(logits_t)
    tok = chosen_tokens[t]
    logp_new = logp_t[tok]
    r_t = torch.exp(logp_new - logprob_old[t])
    unclipped = r_t * advantages[t]
    clipped_r = torch.clamp(r_t, 1.0 - eps_clip, 1.0 + eps_clip)
    clipped = clipped_r * advantages[t]
    total_unclipped = total_unclipped + unclipped
    total_clipped   = total_clipped   + clipped
    print(f"t={t}: tok={tok}  logp_old={logprob_old[t]:+.4f}  logp_new={logp_new:+.4f}  r_t={float(r_t):.4f}  "
          f"uncl={float(unclipped):+.4f}  clp={float(clipped):+.4f}  picked=min(uncl,clp)")


t=0: tok=2  logp_old=-1.7502  logp_new=-1.7502  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)
t=1: tok=1  logp_old=-1.5884  logp_new=-1.5884  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)
t=2: tok=4  logp_old=-1.6374  logp_new=-1.6374  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)


In [60]:
# Promediamos el surrogate (min) y lo negamos para minimizar
surrogate = torch.minimum(total_unclipped, total_clipped) / T
loss = -surrogate
print(f"\nSurrogate (mean): {float(surrogate):+.6f}  ->  PPO policy loss = {float(loss):+.6f}")



Surrogate (mean): +0.400000  ->  PPO policy loss = -0.400000


In [61]:
# Backprop + step
opt.zero_grad()
loss.backward()
opt.step()


In [62]:
# ----------------------------------------------------------
# 4) AFTER update: ver cómo cambiaron P_new de los tokens elegidos
# ----------------------------------------------------------
with torch.no_grad():
    after = []
    ratio_after = []
    for t in range(T):
        logits_t = model(I[t])
        logp_t  = log_softmax(logits_t)
        tok = chosen_tokens[t]
        logp_new = logp_t[tok]
        r_t = torch.exp(logp_new - logprob_old[t])
        after.append((tok, float(logp_new), float(logp_t.exp()[tok])))
        ratio_after.append(float(r_t))

print("\nPer-step chosen token probabilities (before -> after):")
for t, ((tok_b, logpb, pb), (tok_a, logpa, pa)) in enumerate(zip(before, after)):
    assert tok_b == tok_a
    arrow = "↑" if pa > pb else ("↓" if pa < pb else "→")
    print(f"  t={t}: token={tok_b}   P_before={pb:.4f} -> P_after={pa:.4f}   ({arrow})")

print("r_t AFTER         :", [f"{r:.4f}" for r in ratio_after])



Per-step chosen token probabilities (before -> after):
  t=0: token=2   P_before=0.1737 -> P_after=0.1839   (↑)
  t=1: token=1   P_before=0.2043 -> P_after=0.2153   (↑)
  t=2: token=4   P_before=0.1945 -> P_after=0.2052   (↑)
r_t AFTER         : ['1.0583', '1.0540', '1.0552']


**PPO**: 

usa un reward model aparte. 


**GRPO**:

**DPO**:

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

torch.manual_seed(0)

vocab_size=5
T=3
lr=0.5

model = nn.Linear(T, vocab_size, bias=False)
with torch.no_grad():
    model.weight.copy_(0.1 * torch.randn(vocab_size, T))
log_softmax = nn.LogSoftmax(dim=-1)

I = torch.eye(T)  # shape (T, T)

logits_t = model(I)          # (vocab,)
print(logits_t)

logp_t  = log_softmax(logits_t) # (vocab,)
print(logp_t)

probs = logp_t.exp()
print(probs)

probs.sum(dim=-1)

tensor([[ 0.0604,  0.0880, -0.0723,  0.0160,  0.2263],
        [ 0.0811,  0.1048,  0.2866, -0.0025, -0.0918],
        [-0.0045, -0.0045, -0.0566,  0.1074, -0.0225]], grad_fn=<MmBackward0>)
tensor([[-1.6176, -1.5900, -1.7502, -1.6619, -1.4517],
        [-1.6121, -1.5884, -1.4065, -1.6957, -1.7849],
        [-1.6194, -1.6193, -1.6714, -1.5075, -1.6374]],
       grad_fn=<LogSoftmaxBackward0>)
tensor([[0.1984, 0.2039, 0.1737, 0.1898, 0.2342],
        [0.1995, 0.2043, 0.2450, 0.1835, 0.1678],
        [0.1980, 0.1980, 0.1880, 0.2215, 0.1945]], grad_fn=<ExpBackward0>)


tensor([1., 1., 1.], grad_fn=<SumBackward1>)